# Speed tests

In [ ]:
from expelliarmus import Wizard
import pathlib
import h5py
import numpy as np
import timeit

In [ ]:
REPEAT= 10

files = ("potter/spinner.dat", "potter/spinner.raw", "potter/pedestrians.raw")
encodings = ("dat", "evt2", "evt3")

for f, encoding in zip(files, encodings):
    print("="*50)
    wizard = Wizard(encoding=encoding)
    fpath = pathlib.Path(f)
    h5_fpath = pathlib.Path(f"./ref_HDF5_{encoding.upper()}.hdf5")
    h5_LZF_fpath = pathlib.Path(f"./ref_HDF5_LZF_{encoding.upper()}.hdf5")
    np_fpath = pathlib.Path(f"./ref_np_{encoding.upper()}.npy")

    arr = wizard.read(fpath)

    # HDF5 
    h5_fp = h5py.File(h5_fpath, "w")
    arr_h5 = h5_fp.create_dataset("arr", arr.shape, arr.dtype)
    arr_h5[:] = wizard.read(fpath)
    h5_fp.close()

    # HDF5 LZF
    h5_LZF_fp = h5py.File(h5_LZF_fpath, "w")
    arr_h5_lzf = h5_LZF_fp.create_dataset("arr", arr.shape, arr.dtype, compression="lzf")
    arr_h5_lzf[:] = wizard.read(fpath)
    h5_LZF_fp.close()
    
    # NumPy
    np.save(np_fpath, arr, allow_pickle=False)

    avg_time_ours = timeit.repeat(lambda: wizard.read(fpath), number=1, repeat=REPEAT) 
    avg_time_ours = sum(avg_time_ours) / len(avg_time_ours)
    print(f"{encoding.upper()} ({fpath.stat().st_size//(1024*1024)}MB) avg_time = {avg_time_ours:.3f} s.")

    h5_fp = h5py.File(h5_fpath)
    avg_time = timeit.repeat(lambda: h5_fp["arr"][:], number=1, repeat=REPEAT) 
    avg_time = sum(avg_time) / len(avg_time)
    print(f"HDF5 {encoding.upper()} ({h5_fpath.stat().st_size//(1024*1024)}MB) avg_time = {avg_time:.3f} s ({'+' if avg_time_ours<avg_time else '-'}{((1-avg_time_ours/avg_time) if avg_time_ours<avg_time else (1-avg_time/avg_time_ours))*100:.2f}%).")
    h5_fp.close()

    h5_LZF_fp = h5py.File(h5_LZF_fpath)
    avg_time = timeit.repeat(lambda: h5_LZF_fp["arr"][:], number=1, repeat=REPEAT) 
    avg_time = sum(avg_time) / len(avg_time)
    print(f"HDF5 LZF {encoding.upper()} ({h5_LZF_fpath.stat().st_size//(1024*1024)}MB) avg_time = {avg_time:.3f} s ({'+' if avg_time_ours<avg_time else '-'}{((1-avg_time_ours/avg_time) if avg_time_ours<avg_time else (1-avg_time/avg_time_ours))*100:.2f}%).")
    h5_LZF_fp.close()

    avg_time = timeit.repeat(lambda: np.load(np_fpath), number=1, repeat=REPEAT) 
    avg_time = sum(avg_time) / len(avg_time)
    print(f"NumPy {encoding.upper()} ({np_fpath.stat().st_size//(1024*1024)}MB) avg_time = {avg_time:.3f} s ({'+' if avg_time_ours<avg_time else '-'}{((1-avg_time_ours/avg_time) if avg_time_ours<avg_time else (1-avg_time/avg_time_ours))*100:.2f}%).")
